In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Portfolio look-through in LUSID

Demonstrates the use of policies to grant access to portfolios based on their Access Metadata (AMD) in LUSID.

Attributes
----------
entitlements
access metadata
amd
"""

toggle_code("Toggle Docstring")

# Access Metadata entitlements

This notebook demonstrates the use of policies to grant access to portfolios based on their Access Metadata in LUSID. There is an associated Knowledge Base article [here](https://support.lusid.com/knowledgebase/article/KA-01940/en-us).

Table of contents:
1. [Setup](#1.-Setup)
2. [Prepare data](#2.-Prepare-data)    
    2.1 [Create portfolios](#2.1-Create-portfolios)
    2.2 [Create instruments](#2.2-Create-instruments)   
    2.3 [Add transactions to the portfolios](#2.3-Add-transactions-to-the-portfolios)
    2.4 [Add Access Metadata to the portfolios](#2.4-Add-Acces-Metadata-to-the-portfolios)
3. [Demonstrate entitlements](#3.-Demonstrate-entitlements)    
    3.1 [Create a role](#3.1-Create-a-role)    
    3.2 [Create a policy to deny access to externally managed portfolios](#3.2-Create-a-policy-to-deny-access-to-externally-managed-portfolios)
    3.3 [Retrieve portfolios](#3.3-Retrieve-portfolios)

---

## 1. Setup

To start, let's import the libraries and initialise the APIs we'll use in the notebook.

In [2]:
# Import Libraries
import os
import random
import math
import json
import pytz

from datetime import datetime, timedelta
from collections import namedtuple

import finbourne_access
import finbourne_identity
import lusid
import pandas as pd

from finbourne_access.utilities import ApiClientFactory as AccessApiClientFactory
from finbourne_access import models as access_models
from finbourne_identity import models as identity_models
from lusid import models as models
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from IPython.core.display import HTML

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"
identity_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "identity"

access_api_factory = finbourne_access.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    access_url=access_api_url,
    app_name="LusidJupyterNotebook",
)

identity_api_factory = finbourne_identity.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    api_url=identity_api_url,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    lusid_api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10352.0,0.5.3090,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
scope = "AccessMetadaEntitlements"
portfolio_A_code = "AccessMetadataEntitlementsPortfolioCodeA"
portfolio_B_code = "AccessMetadataEntitlementsPortfolioCodeB"
portfolio_A_name = "Access metadata entitlements portfolio A"
portfolio_B_name = "Access metadata entitlements portfolio B"

In [4]:
# Initialise all APIs used in the notebook
transaction_portfolios_api = lusid_api_factory.build(lusid.TransactionPortfoliosApi)
portfolios_api = lusid_api_factory.build(lusid.PortfoliosApi)
instruments_api = lusid_api_factory.build(lusid.InstrumentsApi)
properties_api = lusid_api_factory.build(lusid.PropertyDefinitionsApi)

policies_api = access_api_factory.build(finbourne_access.PoliciesApi)
access_roles_api = access_api_factory.build(finbourne_access.RolesApi)

identity_roles_api = identity_api_factory.build(finbourne_identity.RolesApi)
users_api = identity_api_factory.build(finbourne_identity.UsersApi)

---

## 2. Prepare data

To demonstrate the entitlements, let's first prepare the data. 

Below, we create two new TransactionPortfolio's denominated in GBP, as well as a set of five instruments identified by their FIGIs.

### 2.1 Create portfolios

In [5]:
def create_portfolio(portfolio_name, portfolio_code):
    # Create the portfolio request.
    create_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_code,
        base_currency="GBP"
    )

    try:
        # Make the call to the API.
        transaction_portfolios_api.create_portfolio(scope, create_portfolio_request)
    except lusid.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 112:  # PortfolioWithIdAlreadyExists
            raise e

create_portfolio(portfolio_A_name, portfolio_A_code)
create_portfolio(portfolio_B_name, portfolio_B_code)

### 2.2 Create instruments

In [6]:
InstrumentSpec = namedtuple("InstrumentSpec", ["Figi", "Name"])

instruments = [
    InstrumentSpec("BBG000FD8G46", "HISCOX LTD"),
    InstrumentSpec("BBG000DW76R4", "ITV PLC"),
    InstrumentSpec("BBG000PQKVN8", "MONDI PLC"),
    InstrumentSpec("BBG000BDWPY0", "NEXT PLC"),
    InstrumentSpec("BBG000BF46Y8", "TESCO PLC"),
]

instruments_to_create = {
    i.Figi: models.InstrumentDefinition(
        name=i.Name, identifiers={"Figi": models.InstrumentIdValue(value=i.Figi)}
    )
    for i in instruments
}

response = instruments_api.upsert_instruments(request_body=instruments_to_create)

instrument_ids = sorted([i.lusid_instrument_id for i in response.values.values()])

### 2.3 Add transactions to the portfolios

The amounts and considerations here are randomised, but typically they would be loaded in externally (for example, from a .csv file).

In [7]:
def load_transactions(portfolio_code):
    # Prepare transaction requests
    transactions = [
        models.TransactionRequest(
            transaction_id=f"TransactionId_{_id}",
            type="Buy",
            instrument_identifiers={"Instrument/default/LusidInstrumentId": _id},
            transaction_date=datetime.now(pytz.UTC).isoformat(),
            settlement_date=(datetime.now(pytz.UTC) + timedelta(days=2)).isoformat(),
            units=math.floor(random.random() * 100),
            total_consideration=lusid.CurrencyAndAmount(
                math.floor(random.random() * 1000), "GBP"
            ),
        )
        for _id in instrument_ids
    ]

    transaction_portfolios_api.upsert_transactions(
        scope, portfolio_code, transactions
    )

load_transactions(portfolio_A_code)
load_transactions(portfolio_B_code)

### 2.4 Add Access Metadata to the portfolios

We want to add Access Metadata to the portfolios as well as a corresponding property. We add a corresponding property as properties are visible in the UI while Access Metadata is not.

In [8]:
# Add access metadata to the portfolios for key IsExternallyManaged
metadata_key = "IsExternallyManaged"

def add_access_metadata(value, portfolio_code):
    upsert_portfolio_access_metadata_request = models.UpsertPortfolioAccessMetadataRequest(
        metadata=[
            models.AccessMetadataValue(
                value=value,
                provider="SampleNotebook")
        ])

    portfolios_api.upsert_portfolio_access_metadata(
        scope,
        portfolio_code,
        metadata_key,
        upsert_portfolio_access_metadata_request)

add_access_metadata("true", portfolio_A_code)
add_access_metadata("false", portfolio_B_code)

To add a property, we first create a property definition (the `PropertyDefinition` schema in the [API](https://www.lusid.com/api/swagger/index.html)). This gives LUSID information about the property type.

We then add a value to this new property for the portfolios.

In [9]:
# Create a property definition for key Portfolio/AccessMetadataEntitlements/IsExternallyManaged
property_domain = "Portfolio"
property_code = "IsExternallyManaged"

try:
    properties_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain=property_domain,
            scope=scope,
            code=property_code,
            display_name=property_code,
            life_time="Perpetual",
            value_required=False,
            data_type_id=models.resource_id.ResourceId(
                scope="system",
                code="string",
            )
        )
    )
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 124:  # "PropertyAlreadyExists"
        raise e

def add_property_value(value, portfolio_code):
    perpetual_property = models.PerpetualProperty(
        f"{property_domain}/{scope}/{property_code}",
        models.PropertyValue(label_value=value)
    )

    # Add the property to the portfolios.
    portfolios_api.upsert_portfolio_properties(
            scope=scope,
            code=portfolio_code,
            request_body={f"{property_domain}/{scope}/{property_code}": perpetual_property},
        )

# Add the property to the portfolios.
add_property_value("true", portfolio_A_code)
add_property_value("false", portfolio_B_code)

---

## 3. Demonstrate entitlements

To demonstrate entitlements, we'll need to have access to two users:
- User A has admin rights and can see both portfolios regardless of whether they are externally managed. We will need this user to grant policies.
- User B has the minimum required rights to see portfolios. Originally user B can see both portfolio A and B.

The process is as follows:
- as User B, try to retrieve the portfolios in the `AccessMetadaEntitlements` scope. This should return two portfolios, `AccessMetadataEntitlementsPortfolioCodeA` and `AccessMetadataEntitlementsPortfolioCodeB`.
- as User A, deny User B access to externally managed portfolios by applying a policy to their role.
- as User B, try retrieving the portfolios in the `AccessMetadaEntitlements` scope again. This should return only the non externally managed portfolios, `AccessMetadataEntitlementsPortfolioCodeB` in this case.

### 3.1 Create a role

Here, we create a new role with two policies applied by default:
- `allow-standard-lusid-features-access`, which gives a user access to standard LUSID features. This policy ships with LUSID out of the box.
- `allow-scope-access-AccessMetadaEntitlements`, which gives a user access to the scope in which our portfolios sit. Without this, the user would not be able to see anything in the `AccessMetadaEntitlements` scope. We'll have to create this using the API.

In [10]:
# WhenSpec objects specify the "lifetime" of a modification:
# when it is activated and when it is deactivated.
when_spec = access_models.WhenSpec(
    activate=datetime.now(tz=pytz.utc) - timedelta(days=2),
    deactivate=datetime(9999, 12, 31, tzinfo=pytz.utc),
)

Here we create a policy to grant access to the scope containing the TransactionPortfolio's, as described above.

In [11]:
allow_scope_policy_code = f"allow-scope-access-{scope}"

# Get access path where to apply the policy.
portfolio_selector_definition = access_models.IdSelectorDefinition(
    identifier={"scope": scope, "code": "AccessMetadataEntitlementsPortfolioCode*"},
    actions=[
        access_models.ActionId(scope="default", activity="Any", entity="Portfolio")
    ],
)

allow_portfolio_path = access_models.SelectorDefinition(
    id_selector_definition=portfolio_selector_definition
)
allow_portfolio_policy_request = access_models.PolicyCreationRequest(
    code=allow_scope_policy_code,
    applications=["LUSID"],
    grant=access_models.Grant.ALLOW,
    selectors=[allow_portfolio_path],
    when=when_spec,
)

try:
    # Create the policy.
    policies_api.create_policy(allow_portfolio_policy_request)
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] not in [612,613,615]:  # PolicyWithCodeAlreadyExists
        raise e

One implementation detail for LUSID roles is that we'll have to create the same role twice: once using the identity API and once using the access API. This is such that the access module, which handles applying policies to a role, can communicate with the identity module, which handles applying roles to users.

In [12]:
role_code = "AccessMetadataEntitlementsRole"
allow_features_policy_code = "allow-standard-lusid-features-access"

# Create the role using the access API.
role_creation_request = access_models.RoleCreationRequest(
    code=role_code,
    description=role_code,
    resource=access_models.RoleResourceRequest(
        policy_id_role_resource=access_models.PolicyIdRoleResource(
            # Here, we apply the two default policies when creating the role.
            policies=[
                access_models.PolicyId(scope="default", code=allow_features_policy_code),
                access_models.PolicyId(
                    scope="default",
                    code=allow_scope_policy_code,
                ),
            ]
        )
    ),
    when=when_spec,
)


try:
    response = access_roles_api.create_role(role_creation_request)
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail['code'] not in [612,613,615]:  # RoleWithCodeAlreadyExists
        raise e

In [13]:
# Create the same role using the identity API.
try:
    response = identity_roles_api.create_role(
        create_role_request=identity_models.CreateRoleRequest(role_code)
    )
except finbourne_identity.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 157:  # RoleWithCodeAlreadyexists
        raise e

Let's assign the newly created role to a user (User B in our scenario).

In [14]:
first_name = "John"
last_name = "Doe"
email_address = "John.Doe@example.com"
login = "AccessMetadataEntitlements@example.com"

The following cell will create a user in your environment if the commented out line is uncommented (the comment is denoted by a `#` at the start of the line). If the cell is run to create a user, the user will need to be activated by following the instructions sent via email to the `email_address` email address.

In [15]:
create_user_request = identity_models.CreateUserRequest(
    first_name=first_name,
    last_name=last_name,
    email_address=email_address,
    login=login,
    type="Personal"
)

#users_api.create_user(create_user_request)

To assign the role to the user, we need to call two endpoints of the identity API:
- `UpdateUsers` in the Users API
- `AddUserToRole` in the Roles API

In [16]:
# Find the user id for this user.
user_id = [user.id for user in users_api.list_users() if user.login == login][0]

# Find the role id for this role.
role_id = [role.id for role in identity_roles_api.list_roles() if role.role_id.code == role_code][0]

response = users_api.update_user(
    id=user_id,
    update_user_request=identity_models.UpdateUserRequest(
        first_name=first_name,
        last_name=last_name,
        email_address=email_address,
        login=login,
        roles=[
            identity_models.RoleId(scope="default", code=role_code),
        ]
    ),
)

identity_roles_api.add_user_to_role(role_id, user_id)

### 3.2 Create a policy to deny access to externally managed portfolios

Here we create a policy that denies access to portfolios if they have a Access Metadata with a key of `IsExternallyManaged` and a value of `"true"`.

In [17]:
access_metadata_policy_code = f"deny-externally-managed-portfolios-access-in-{scope}"

metadata_selector_definition = access_models.MetadataSelectorDefinition(
    expressions=[
        access_models.MetadataExpression(
            metadata_key=metadata_key,
            operator=access_models.Operator.EQUALS,
            text_value="true")
    ],
    actions=[
        access_models.ActionId(scope="default", activity="Any", entity="Portfolio")
    ],
    name="Access Metadata selector",
    description="Access Metadata selector"
)

allow_property_path = access_models.SelectorDefinition(
    metadata_selector_definition=metadata_selector_definition
)

allow_property_policy_request = access_models.PolicyCreationRequest(
    code=access_metadata_policy_code,
    description=f"Denies access to externally managed portfolios in {scope} scope",
    applications=["LUSID"],
    grant=access_models.Grant.DENY,
    selectors=[allow_property_path],
    when=when_spec
)

try:
    # Create the policy.
    policies_api.create_policy(allow_property_policy_request)
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail['code'] not in [612,613,615]:  # PolicyWithCodeAlreadyExists
        raise e

### 3.3 Retrieve portfolios

First, let's try to retrieve the portfolios with User B's current entitlements. To do so, run the following statement as User B.

In [5]:
lusid_response_to_data_frame(portfolios_api.list_portfolios_for_scope(scope))

,href,id.scope,id.code,type,display_name,created,version.effective_from,version.as_at_date,is_derived,base_currency,properties,relationships,instrument_scopes,accounting_method,amortisation_method,links.0.relation,links.0.href,links.0.method,links.1.relation,links.1.href,links.1.method,links.2.relation,links.2.href,links.2.method,links.3.relation,links.3.href,links.3.method,links.4.relation,links.4.href,links.4.method,links.5.relation,links.5.href,links.5.method
0,https://fbn-ci.lusid.com/api/api/portfolios/Ac...,AccessMetadaEntitlements,AccessMetadataEntitlementsPortfolioCodeB,Transaction,Access metadata entitlements portfolio B,2022-11-28 10:34:57.901278+00:00,2022-11-28 16:27:06.487977+00:00,2022-11-28 16:27:06.380694+00:00,False,GBP,{},[],[],Default,NoAmortisation,Properties,https://fbn-ci.lusid.com/api/api/portfolios/Ac...,GET,Commands,https://fbn-ci.lusid.com/api/api/portfolios/Ac...,GET,Details,https://fbn-ci.lusid.com/api/api/transactionpo...,GET,Transactions,https://fbn-ci.lusid.com/api/api/transactionpo...,GET,Holdings,https://fbn-ci.lusid.com/api/api/transactionpo...,GET,HoldingsAdjustments,https://fbn-ci.lusid.com/api/api/transactionpo...,GET


To entitle User B, apply the Access Metadata policy we created above to their role by running the below statement as User A.

In [19]:
role_update_request = access_models.RoleUpdateRequest(
    description="Add new Access Metadata policy",
    resource=access_models.RoleResourceRequest(
        policy_id_role_resource=access_models.PolicyIdRoleResource(
            policies=[
                access_models.PolicyId(scope="default", code=allow_features_policy_code),
                access_models.PolicyId(scope="default", code=allow_scope_policy_code),
                access_models.PolicyId(scope="default", code=access_metadata_policy_code),
            ]
        )
    ),
    when=when_spec,
)

response = access_roles_api.update_role(
    code=role_code, scope="default", role_update_request=role_update_request
)

Finally, let's try retrieving the portfolios as User B. The results now only contain portfolio B.

In [6]:
lusid_response_to_data_frame(portfolios_api.list_portfolios_for_scope(scope))

,href,id.scope,id.code,type,display_name,created,version.effective_from,version.as_at_date,is_derived,base_currency,properties,relationships,instrument_scopes,accounting_method,amortisation_method,links.0.relation,links.0.href,links.0.method,links.1.relation,links.1.href,links.1.method,links.2.relation,links.2.href,links.2.method,links.3.relation,links.3.href,links.3.method,links.4.relation,links.4.href,links.4.method,links.5.relation,links.5.href,links.5.method
0,https://fbn-ci.lusid.com/api/api/portfolios/Ac...,AccessMetadaEntitlements,AccessMetadataEntitlementsPortfolioCodeB,Transaction,Access metadata entitlements portfolio B,2022-11-28 10:34:57.901278+00:00,2022-11-28 16:27:25.845097+00:00,2022-11-28 16:27:24.868495+00:00,False,GBP,{},[],[],Default,NoAmortisation,Properties,https://fbn-ci.lusid.com/api/api/portfolios/Ac...,GET,Commands,https://fbn-ci.lusid.com/api/api/portfolios/Ac...,GET,Details,https://fbn-ci.lusid.com/api/api/transactionpo...,GET,Transactions,https://fbn-ci.lusid.com/api/api/transactionpo...,GET,Holdings,https://fbn-ci.lusid.com/api/api/transactionpo...,GET,HoldingsAdjustments,https://fbn-ci.lusid.com/api/api/transactionpo...,GET
